In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
import re

In [ ]:

data = pd.read_csv('MR1080828.csv')
selector = ['PNO', 'total', 'BED/chair', 'CDATE', 'PLACE',
            'ITEM', 'ORDERDR', 'MD_NO', 'CASENO',
            'AMOUNT', 'IO', 'DEPT', 'Btime', 'Etime',
            'SEX', 'AGE', 'POS', 'MODEL_NAME']

data = data[selector]
data = data.dropna()
data = data[(data['Btime'] != '0:00') & (data['Etime'] != '0:00')]



In [53]:
df = data.copy()

In [54]:
df

,PNO,total,BED/chair,CDATE,PLACE,ITEM,ORDERDR,MD_NO,CASENO,AMOUNT,IO,DEPT,Btime,Etime,SEX,AGE,POS,MODEL_NAME
0,36548159,0:40,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,4063.0,405322.0,332.0,11500.0,O,13,8:15,8:55,F,45.0,41.0,Optima MR450w
1,38313603,0:45,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,4370.0,405322.0,72.0,11500.0,O,52,09:00,9:45,M,50.0,62.0,Optima MR450w
2,17627227,0:50,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,4196.0,405322.0,5182.0,11500.0,O,20,9:50,10:40,M,63.0,1.0,Optima MR450w
3,48747186,0:35,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,6272.0,405322.0,92.0,11500.0,O,14,10:45,11:20,F,64.0,1.0,Optima MR450w
4,40667757,0:35,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,4106.0,405322.0,1232.0,13800.0,E,1,11:25,12:00,M,75.0,1.0,Optima MR450w
5,50381299,0:40,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70004.0,4355.0,405322.0,532.0,11500.0,O,53,13:05,13:45,M,74.0,1.0,Optima MR450w
6,29328867,0:30,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70003.0,4263.0,405322.0,6292.0,6500.0,O,53,13:50,14:20,M,50.0,34.0,Optima MR450w
7,28112492,0:30,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70003.0,5970.0,405322.0,5042.0,6500.0,O,14,14:25,14:55,F,41.0,19.0,Optima MR450w
8,117230,0:35,normal,2017/10/2 上午 12:00:00,ROOM18(馬3F),70003.0,4370.0,405322.0,7542.0,6500.0,O,52,15:35,16:10,F,55.0,65.0,Optima MR450w
9,35912660,0:30,normal,2017/10/3 上午 12:00:00,ROOM18(馬3F),70003.0,4379.0,405322.0,722.0,6500.0,O,53,8:10,8:40,F,64.0,34.0,Optima MR450w


In [55]:
# def convert_type(df) -> df:
df['CDATE'] = df['CDATE'].astype(str)
df['DATE'] = df['CDATE'].apply(lambda s: s.split(' ')[0])
# 建立日期(原本的CDATE: 2017/10/2 上午 12:00:00)

timefeatures = ['Btime', 'Etime']
for t in timefeatures:
    df[t] = df[t].str.replace(';', ':')
    df[t] = pd.to_datetime(df[t], format="%H:%M")

# def adjust_time(df):
# 只調整這四筆資料
df.loc[619, 'Btime'] = datetime.strptime('1900-01-01 10:15:00', '%Y-%m-%d %H:%M:%S')
df.loc[3181, 'Btime'] = datetime.strptime('1900-01-01 20:25:00', '%Y-%m-%d %H:%M:%S')
df.loc[1238, 'Etime'] = datetime.strptime('1900-01-01 10:00:00', '%Y-%m-%d %H:%M:%S')
df.loc[1573, 'Btime'] = datetime.strptime('1900-01-01 09:20:00', '%Y-%m-%d %H:%M:%S')


# 挑出同一天內檢查時間有重疊的資料
df = df[df['DATE'] != '2018/5/5']

# 把時間單位改成秒
df['total'] = (df['Etime'] - df['Btime']) / np.timedelta64(1, 's')
df = df[df['total'] > 0]


# 以下有點問題 25% 的部分含有一半的資料
# def label_time(t):
#     if t >= 2700:
#         return '75%'
#     elif t >= 2400:
#         return '50%'
#     else:
#         return '25%'
def better_label_time(t):
    if t >= 2700:
        return "75%"
    elif t>=2400:
        return "50%"
    elif t>=1800:
        return "25%"
    else:
        return "0%"
    
df['time_label'] = df['total'].apply(better_label_time)

df['YM'] = df['CDATE'].apply(lambda s: s.split('/')[0] + '/' + s.split('/')[1])


c:\users\leechiente\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [58]:
df.to_csv("cleaned_data_20190906_JuiLee.csv", encoding="utf-8", index=False)

In [ ]:
if __name__ == "__main__":
    # 原始時間以及預測時間
    data = get_data() #篩選需要的欄位
    data = convert_type(data) #調整部分變數型態
    
    # 以上完成破譯
    
    submission = build_xgb(data) #mape 0.19
    # use_old_xgb(data)
    # rulemining(data) #新變數沒有相關的法則

    # 觀察每天的變化
    job = make_duty_table(data)
    # print(job[['PNO', 'worktime']].describe()) # 醫護人員預估範圍:53~40, mean=48
    # plot_efficiency(job)
    # job = make_duty_table(submission)
    # plot_efficiency(job)
    # plot_wt()

    # 排程效果
    # hospital_schedule(job)
    # compare_before_after()
    # print(job[job['mdtime'] / job['worktime'] < 0.8]['DATE']) # 找出使用效率不高的那天
    # print(job[job['worktime'] > 9*3600][['worktime', 'DATE']]) # 找出加班的那一天
    plot_one_day()
